
# 注意メカニズムとトランスフォーマー

:label: `chap_attention-and-transformers`

深層学習ブームの初期は、主に多層パーセプトロン、畳み込みネットワーク、リカレント ネットワーク アーキテクチャを使用して生成された結果によって推進されました。注目すべきことに、2010 年代のディープラーニングの躍進の多くを支えたモデル アーキテクチャは、30 年近くが経過したにもかかわらず、以前のものと比べて驚くほどほとんど変わっていませんでした。多くの新しい方法論的革新がほとんどの実践者のツールキットに取り入れられましたが、ReLU アクティベーション、残差層、バッチ正規化、ドロップアウト、適応学習率スケジュールが思い浮かびますが、コアとなる基礎的なアーキテクチャはスケールアップされた実装として明確に認識できました。古典的なアイデアの。別のアイデアを提案した何千もの論文にもかかわらず、古典的な畳み込みニューラル ネットワーク (:numref: `chap_cnn` ) に似たモデルがコンピューター ビジョンの*最先端の地位*を維持し、LSTM リカレント ニューラル ネットワーク (:numref: `sec_lstm` ) のゼップ ホッホライターのオリジナル設計に似たモデルが優勢でした。自然言語処理のほとんどのアプリケーションに適用されます。おそらく、その時点まで、ディープラーニングの急速な出現は、利用可能な計算リソースの変化 (GPU による並列コンピューティングの革新による) と、大量のデータ リソースの利用可能性 (安価なストレージとインターネット サービスによる) が主な原因であるように見えました。 。確かにこれらの要因が、このテクノロジーの威力増大の主な要因であることに変わりはありませんが、私たちは、ついに、主要なアーキテクチャの状況に大きな変化が起きていることも目の当たりにしています。

現時点では、ほぼすべての自然言語処理タスクの主要なモデルは、Transformer アーキテクチャに基づいています。自然言語処理における新しいタスクが与えられた場合、デフォルトの最初のアプローチは、大規模な Transformer ベースの事前トレーニング済みモデルを取得することです (例: BERT :cite: `Devlin.Chang.Lee.ea.2018` 、 ELECTRA :cite: `clark2019electra` 、 RoBERTa :cite: `liu2019roberta` 、または Longformer :cite: `beltagy2020longformer` ) 必要に応じて出力層を適応させ、下流のタスクに使用できるデータに基づいてモデルを微調整します。 OpenAI の大規模言語モデルを中心としたここ数年間の息を呑むようなニュース報道に注意を払ってきたなら、あなたは GPT-2 および GPT-3 Transformer ベースのモデルを中心とした会話を追跡していることになります (引用: `Radford.Wu.Child.ea.2019,brown2020language` 。一方、ビジョン Transformer は、画像認識、オブジェクト検出、セマンティック セグメンテーション、超解像度など、さまざまなビジョン タスクのデフォルト モデルとして登場しました:cite: `Dosovitskiy.Beyer.Kolesnikov.ea.2021,liu2021swin` 。トランスフォーマーは、音声認識 :cite: `gulati2020conformer` 、強化学習 :cite: `chen2021decision` 、グラフ ニューラル ネットワーク :cite: `dwivedi2020generalization`の競合手法としても登場しました。

 Transformer モデルの背後にある中心的なアイデアは、*アテンション メカニズム*です。これは、機械翻訳のようなシーケンス間アプリケーションに適用されるエンコーダー デコーダー RNN の拡張機能として当初構想されていた革新です :cite: `Bahdanau.Cho.Bengio.2014` 。機械翻訳の最初のシーケンスツーシーケンス モデル :cite: `Sutskever.Vinyals.Le.2014`では、入力全体がエンコーダーによって単一の固定長ベクトルに圧縮され、デコーダーに供給されたことを思い出してください。注意の背後にある直観は、入力を圧縮するよりも、デコーダが各ステップで入力シーケンスを再検討する方が良いのではないかということです。さらに、入力の同じ表現を常に見るのではなく、デコーダが特定のデコード ステップで入力シーケンスの特定の部分に選択的に焦点を当てる必要があると考える人もいるかもしれません。 Bahdanau のアテンション メカニズムは、デコーダが各デコード ステップで入力のさまざまな部分に動的に*対応*できる簡単な手段を提供しました。大まかな考え方は、エンコーダが元の入力シーケンスと等しい長さの表現を生成できるということです。次に、デコード時に、デコーダは (何らかの制御メカニズムを介して) 各タイム ステップでの入力上の表現の重み付けされた合計から構成されるコンテキスト ベクトルを入力として受け取ることができます。直観的には、重みは各ステップのコンテキストが各入力トークンに「焦点を当てる」程度を決定します。重要なのは、重みを割り当てるこのプロセスを微分可能にして、他のすべてのニューラル ネットワーク パラメーターとともに学習できるようにすることです。

当初、このアイデアは、すでに機械翻訳アプリケーションを支配していたリカレント ニューラル ネットワークの機能強化として非常に成功しました。このモデルは、元のエンコーダ/デコーダのシーケンス間アーキテクチャよりも優れたパフォーマンスを示しました。さらに、研究者らは、注意の重みのパターンを調べることで、優れた定性的な洞察が得られることもあると指摘しました。翻訳タスクでは、注意モデルは、ターゲット言語で対応する単語を生成するときに、言語をまたがる同義語に高い注意の重みを割り当てることがよくありました。たとえば、「足が痛い」という文を「j&#39;ai mal au pieds」に翻訳する場合、ニューラル ネットワークは、対応するフランス語の単語「pieds」を生成するときに、「足」の表現に高い注意の重みを割り当てる可能性があります。これらの洞察は、注意モデルが「解釈可能性」を与えるという主張に拍車をかけましたが、注意の重みが正確に何を意味するのか、つまり、注意の重みをどのように*解釈す*べきかは、依然として曖昧な研究課題のままです。

しかし、注意メカニズムは、エンコーダ/デコーダリカレント ニューラル ネットワークの拡張としての有用性や、顕著な入力を選択するための推定上の有用性を超えて、より重要な懸念事項としてすぐに浮上しました。 2017 年、:citet: `Vaswani.Shazeer.Parmar.ea.2017` 、機械翻訳用の Transformer アーキテクチャを提案しました。これは、相互接続の繰り返しを省略し、代わりに、入出力トークン間のすべての関係を捕捉するために、巧妙に配置されたアテンション メカニズムに依存します。このアーキテクチャは非常に優れたパフォーマンスを示し、2018 年までに Transformer は最先端の自然言語処理システムの大部分に採用されるようになりました。さらに、同時に、自然言語処理における主流の手法は、自己教師ありの事前トレーニング目標を最適化するために、膨大な汎用バックグラウンド コーパスで大規模なモデルを事前トレーニングし、その後、利用可能な下流データを使用してこれらのモデルを微調整することになりました。 Transformer と従来のアーキテクチャの間のギャップは、この事前トレーニング パラダイムに適用された場合に特に拡大しました。そのため、Transformer の隆盛は、現在では*基礎モデル*と呼ばれることもある、そのような大規模な事前トレーニング済みモデルの隆盛と一致しました。 :cite: `bommasani2021opportunities` 。

この章では、最も基本的な直感とアイデアの最も単純な具体化から始めて、注意モデルを紹介します。次に、Transformer アーキテクチャ、ビジョン Transformer、および最新の Transformer ベースの事前トレーニング済みモデルの状況に到達します。

 :begin_tab:toc
- [クエリ-キー-値](queries-keys-values.ipynb)
- [注意力のプール](attention-pooling.ipynb)
- [注意スコアリング関数](attention-scoring-functions.ipynb)
- [バダナウ-注意](bahdanau-attention.ipynb)
- [マルチヘッドアテンション](multihead-attention.ipynb)
- [セルフアテンションと位置エンコーディング](self-attention-and-positional-encoding.ipynb)
- [変成器](transformer.ipynb)
- [ビジョントランスフォーマー](vision-transformer.ipynb)
- [大規模な事前トレーニングトランスフォーマー](large-pretraining-transformers.ipynb):end_tab:
